### Démarrage de tensorboard et imports principaux

In [1]:
# Agrandir le notebook ?
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

# Démarrage de tensorboard pour notebook
%load_ext tensorboard

import sys
from matplotlib import pyplot
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import *
from tensorflow.keras.models import *
from tensorflow.keras import regularizers
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *
import numpy as np
import datetime
import os
import pandas as pd
from PIL import Image
import shutil  
from math import ceil, floor

from tensorflow.python.framework import ops #pour tenter de reset tensorboard, sans grand succès
ops.reset_default_graph()

In [11]:
# Parametres de verification : ==> adjust

list_indiv_id = ['1', '2']
list_epochs = [10, 10]
list_depth = [6, 2]
list_l1 = [0.001, 0]
list_l2 = [0, 0.001]
list_dropout = [0, 0.2]
list_filters_per_layers = [1024, 1024]
list_activations = ["relu", "relu"]

# Chemin dossier courant
LOGS_DIRECTORY = os.getcwd() + "\\logs\\resnets\\logs_" + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M") #dossier de la session
print(LOGS_DIRECTORY)

D:\Projets\TestsDL\logs\resnets\logs_2020-01-28_18-43


### Fonctions pour préparer le dataset

In [3]:
# Chargement des datasets de train et de validation + one hot encoding
def load_dataset():
    # Chargement des données cifar10
    (trainX, trainY), (testX, testY) = cifar10.load_data()
    # one hot encode encoding sur les labels
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)  
    
    Xtrain = np.reshape(np.asarray([np.mean(im, axis=2, keepdims=True) for im in trainX]),(50000,32,32))
    Xtest = np.reshape(np.asarray([np.mean(im, axis=2, keepdims=True) for im in testX]),(10000,32,32))
        
    
    return Xtrain, trainY, Xtest, testY

# Normalisation pour accroître la vitesse du modèle (en redimensionnant les pixels)
def prep_pixels(train, test):
    # Convertion des int en float
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    # Normalisation pour avoir des nombres entre 0 et 1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    # Retourner les images normalisées
    return train_norm, test_norm


### Classe Python pour définir les individus

In [9]:
# Classe pour les convnets
class IndividuConvnets:
    def __init__(self, indiv_id='1', epochs=10, depth=34, l1=0, l2=0, dropout=0, filters_per_layers=64, activation):
        # Initialisation de nos variables
        self.time_fit = datetime.datetime.now()
        self.my_reguralizer = None

        if depth <= 0:
            self.depth = 1
        else:
            self.depth = depth
        self.loss = 0
        self.accuracy = 0
        self.indiv_id = indiv_id
        self.epochs = epochs
        self.activation = activation

        # On peut mettre l1 et l2 en même temps mais pour l'instant on le gère pas
        if (l1 != 0 and l2 != 0):
            self.l1 = 0
            self.l2 = 0
        else:
            self.l1 = l1
            self.l2 = l2

        self.dropout = dropout
        self.filters_per_layers = filters_per_layers
    
    # ToString()
    def __str__(self):  # ToString
        ma_liste = []
        ma_liste.append("indiv_id:{},\n ".format(self.indiv_id))
        ma_liste.append("epochs:{},\n ".format(self.epochs))
        ma_liste.append("nb_layers:{},\n ".format(self.depth))
        ma_liste.append("l1:{},\n ".format(self.l1))
        ma_liste.append("l2:{},\n ".format(self.l2))
        ma_liste.append("dropout:{},\n ".format(self.dropout))
        ma_liste.append("filters_per_layers:{},\n ".format(self.filters_per_layers))
        ma_liste.append("activation:{},\n ".format(self.activation))
        
        return ma_liste
    
   
    def create_and_train_model(self, trainX, trainY, testX, testY, use_skip_connections: bool = True):     
        start = datetime.datetime.now()

        # Update indiv_id pour avoir un vrai ID unique
        #self.indiv_id = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        # Choix d'un emplacement pour les logs
        log_dir = LOGS_DIRECTORY + "\\logs_" + self.indiv_id + "\\tensorboard_data\\"
        
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        
        print("log dir = ", log_dir)
        
        # l1 et l2 en même temps dans le modèle pas encore géré
        if self.l1 > 0:
            self.my_regularizer = regularizers.l1(self.l1 / self.depth)
        elif self.l2 > 0:
            self.my_regularizer = regularizers.l2(self.l2 / self.depth)
           
        
        input_layer = Input((32, 32))
        flatten_layer_output = Flatten(name="flatten")(input_layer)

        penultimate_output = None
        last_output = flatten_layer_output        
        
        for i in range(self.depth):
            
            if penultimate_output is not None and use_skip_connections:
                add_output = Add(name=f"Add_{i}")([last_output, penultimate_output])
                penultimate_output = add_output
                last_output = Dense(self.filters_per_layers, activation=linear, name=f"Dense_{i}", kernel_regularizer=self.my_reguralizer)(add_output)
                last_output = Activation(activation=self.activation, name=f"Activation_{i}")(last_output)
            else:
                penultimate_output = last_output
                last_output = Dense(self.filters_per_layers, activation=linear, name=f"Dense_{i}", kernel_regularizer=self.my_reguralizer)(last_output)
                last_output = Activation(activation=self.activation, name=f"Activation_{i}")(last_output)

        if use_skip_connections:
            last_output = Add(name=f"Add_output")([last_output, penultimate_output])
        
        output_tensor = Dense(10, activation=softmax, name=f"Dense_output")(last_output)
        model = tf.keras.Model(input_layer, output_tensor)

        # Compiler le modèle
        model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
        
              
        # Entrainer le modele
        history = model.fit(trainX, trainY, epochs=self.epochs, batch_size=64, validation_data=(testX, testY), verbose=1, callbacks=[tensorboard_callback])
       
        end = datetime.datetime.now()
        self.time_fit = end - start
        print("Time for fit = ", self.time_fit)

        return history, model
    
    
    def save_model(self, history, model):
        
        # Sauvegarde du modèle
        plot_model(model, "model.png")
        
         # Deplacement modele au bon endroit
        main_dir = os.getcwd() + "\\model.png"
        dest = LOGS_DIRECTORY + "\\logs_" + self.indiv_id + "\\model.png"
        shutil.move(main_dir, dest)
        
        # Afficher nos résultats dans un graphique matplotlib sauvegardé
        pyplot.gcf().subplots_adjust(hspace = 0.5)

        # Afficher la loss
        pyplot.subplot(211)
        pyplot.title('Cross Entropy Loss')
        pyplot.plot(history.history['loss'], color='blue', label='train')
        pyplot.plot(history.history['val_loss'], color='orange', label='test')
        
        # Afficher l'accuracy
        pyplot.subplot(212)
        pyplot.title('Classification Accuracy')
        pyplot.plot(history.history['accuracy'], color='blue', label='train')
        pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
        
        # Sauvegarde
        filename = LOGS_DIRECTORY + "\\logs_" + self.indiv_id + "\\"
        pyplot.savefig( filename + 'plot.png')
        pyplot.close()
       
        
        print("LOSS : ", round(history.history['loss'][0].item(), 3))
        print("VAL_LOSS : ", round(history.history['val_loss'][0].item(), 3))
        print("ACCURACY : ", round(history.history['accuracy'][0].item(), 3))
        print("VAL_ACCURACY : ", round(history.history['val_accuracy'][0].item(), 3))
        
        # attributs pour créer les csv indivudels et le csv global
        self.loss = round(history.history['loss'][0].item(), 3)
        self.val_loss = round(history.history['val_loss'][0].item(), 3)
        self.accuracy = round(history.history['accuracy'][0].item(), 3)
        self.val_accuracy = round(history.history['val_accuracy'][0].item(), 3)
        self.time_taken = round(self.time_fit.total_seconds(),2)
        
        # Créer un dataframe pandas (avec hyperparams) et le sauvegarder en CSV
        df = pd.DataFrame({'indiv_id': [self.indiv_id],
                           'epochs': [self.epochs],
                           'depth': [self.depth],
                           'l1': [self.l1],
                           'l2': [self.l2],
                           'dropout': [self.dropout],
                           'filters_per_layers': [self.filters_per_layers],
                           'activation': [self.activation],
                           'loss': [self.loss],
                           'val_loss': [self.val_loss],
                           'accuracy': [self.accuracy],
                           'val_accuracy': [self.val_accuracy],
                           'time_taken' : [self.time_taken]
                          })
        
        df.to_csv(path_or_buf=filename+"recap.csv",index=False)
    
    # Lance toutes les étapes
    def exec_indiv(self, trainX, testX, trainY, testY):    
        # Créer et entrainer le modele    
        
        history, model = self.create_and_train_model(trainX, trainY, testX, testY)
        
        # Sauvegarder le modèle
        save = self.save_model(history, model)

In [ ]:
fullPath = os.getcwd() + "\\model.png", LOGS_DIRECTORY + "\\model.png"
print(fullPath)

### Classe Python qui va démarrer les tests des neural nets


In [5]:
# Classe générale qui va nous servir à effectuer des actions sur des individus
class MyTraining:
    # Prends un ID et une liste d'individus 
    def __init__(self, id_train, indiv_list):
        
        self.id_train = id_train
        self.indiv_list = indiv_list
    
    def train(self):
        
        # Charger les données
        trainX, trainY, testX, testY = load_dataset()
        
        # Normaliser les données
        trainX, testX = prep_pixels(trainX, testX)
        
        print("Start training\n")
        
        for indiv in self.indiv_list:
            print("indiv ", indiv.indiv_id, "\n")
            indiv.exec_indiv(trainX, testX, trainY, testY)
            print("-----------------------------------------------------------------\n")
        # self, indiv_id='1', epochs=10, depth=34, l1=0, l2=0, dropout=0, filters_per_layers=64
        # Fusion des csv  
        merge_csv = pd.DataFrame(columns=['indiv_id', 'epochs', 'depth', 'l1', 'l2', 'dropout', 'filters_per_layers','activation'
                                          'loss', 'val_loss', 'accuracy', 'val_accuracy', 'time_taken'])
        for indiv in self.indiv_list:
            merge_csv = merge_csv.append({'indiv_id': indiv.indiv_id, 'epochs': indiv.epochs,'depth' : indiv.depth,'l1' : indiv.l1,
                                          'l2' : indiv.l2, 'dropout' : indiv.dropout,'filters_per_layers' : indiv.filters_per_layers,
                                          'activation' : self.activation, 'loss' : indiv.loss,'val_loss' : indiv.val_loss, 
                                          'accuracy' : indiv.accuracy, 'val_accuracy' : indiv.val_accuracy,'time_taken' : indiv.time_taken},
                                         ignore_index=True)
        
        # sauvegarde
        merge_csv.to_csv(LOGS_DIRECTORY + "\\combined_recap.csv", index=False)
            
    
    def all_indiv(self):
        
        # Affiche les caractéristiques de l'ensemble des individus
        for indiv in self.indiv_list:
            print('\n'.join(indiv.__str__()))
            for tir in range(80): print('-', end='')
            print()

### Hyper paramètres


In [ ]:
#Premier Test : ==> adjust
# CONSTANTES : nb_layers = 8, batch_size = 50, epochs 100, lr = 0.01, momentum = 0.9, optimizer Adam, 
#padding = same, maxpool, relu, kernel = (3,3)
# 
# * 3 convnets sans regularization et MLP à 128
#   - 2 filters double(2) avec filters (16, 32) 
#   - 1 sans filters double avec filters (32)

# 2 convnets sans regu ou on test le MLP_end et filters 64
#  * 1 sans double, filter 64 avec MLP_end(128)
#  * 1 sans double, filter 64 avec MLP_end(0)

# 5 convnets avec regu (+ MLP à 128) et filters 32 sans double
#  * 1 convnet avec l1 à 0.01
#  * 1 convnet avec l1 à 0.01 et batchnorm
#  * 1 convnet avec l2 à 0.01 et batchnorm
#  * 1 convnet avec L1 et L2 à 0.01 et batchnorm
#  * 1 convnet avec L1 et L2 à 0.01 + batchnorm + dropout à 0.2

# LEXIQUE PARAM : 
# * filters_double permet de savoir toutes les combien de couche on double les filtres, si 0 on double pas

#list_indiv_id = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
#list_epochs = [100, 100, 100, 100, 100, 100, 100, 100, 100, 100]
#list_depth = [8,8,8,8,8,8,8,8,8,8]
#list_l1 = [0, 0, 0, 0, 0, 0.01, 0.01, 0.01, 0.01, 0.01]
#list_l2 = [0, 0, 0, 0, 0, 0, 0, 0.01, 0.01, 0.01]
#list_dropout = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0.2]
#list_filters_per_layers = [16, 32, 32, 64, 64, 32, 32, 32, 32, 32]

#main_directory =("D:\\Projets\\TestsDL"+datetime.datetime.now().strftime("%Y%m%d-%H%M"))


### Traitement général (train de l'ensemble des modèles)

In [12]:
# Création des individus (des neurals nets, ici convnet) ==> adjust
list_indiv = []
for num in range(len(list_indiv_id)):
    list_indiv.append(IndividuConvnets(
        list_indiv_id[num],
          list_epochs[num],
          list_depth[num],
          list_l1[num],
          list_l2[num],
          list_dropout[num],
          list_filters_per_layers[num],
          list_activations[num]
        )
    )

# Chargement de la classe training, affichage des individus et train de tous les convnets
training_1 = MyTraining(1, list_indiv)
training_1.all_indiv()
training_1.train()

indiv_id:1,
 
epochs:10,
 
nb_layers:6,
 
l1:0.001,
 
l2:0,
 
dropout:0,
 
filters_per_layers:1024,
 
--------------------------------------------------------------------------------
indiv_id:2,
 
epochs:10,
 
nb_layers:2,
 
l1:0,
 
l2:0.001,
 
dropout:0.2,
 
filters_per_layers:1024,
 
--------------------------------------------------------------------------------
Start training

indiv  1 

log dir =  D:\Projets\TestsDL\logs\resnets\logs_2020-01-28_18-43\logs_1\tensorboard_data\
Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 11s 217us/sample - loss: 2.7133 - accuracy: 0.2424 - val_loss: 1.9533 - val_accuracy: 0.2897
Epoch 2/10
50000/50000 [==============================] - 9s 189us/sample - loss: 1.9309 - accuracy: 0.3085 - val_loss: 1.9191 - val_accuracy: 0.3257
Epoch 3/10
50000/50000 [==============================] - 8s 158us/sample - loss: 1.8641 - accuracy: 0.3345 - val_loss: 1.8227 - val_accuracy: 0.3451
Epoch 4/10
500

### Partie tensorboard

In [ ]:
# Procedure pour utiliser tensorboard
#  1 load la première cell
#  2 utiliser la derniere cell avec --logdir (précisez bien votre répertoire, plus sur que ça
#    fonctionne avec une string "mon_path"
#  3 Vous NE POURREZ PLUS update tensorboard sur ce port et il y aura des bugs, pour éviter ça
#    quand vous voulez faire une update, fermez jupyter notebook (shutdown total) et réouvrez le 
#    OU, faites kernel->interrupt et changez de port + de folder de log

#si vous voulez tenter de tuer des process
#os.system("taskkill /im tensorboard.exe /f") #kill tous les processus qui utilisent tensorboard
#os.system('!kill 18776') #kill le processus X

In [ ]:
# Liste des ports utilisés par tensorboard, attention ça se remplit vite et il faut kill jupyter pour clean
from tensorboard import notebook
notebook.list()

In [ ]:
# Code pour démarrer tensorboard dans le dossier souhaité [PRECISEZ BIEN LE DOSSIER ICI]
%tensorboard --logdir "CONVNETS_20200116-2354\\logs_20200116-235500\\tensorboard_data" --port=6152

In [ ]:
# Si vous avez la folie des grandeurs
notebook.display(port=6066, height=1000) 

In [ ]:
### Fichier CSV combined_recap + Graphique

In [ ]:
# Commandes pandas utiles

#data_csv = pd.read_csv(main_directory+"\\combined_recap.csv")

#meilleure accuracy, moins pire loss par ex
#data_csv.sort_values(["elapsed"], axis=0, 
                 #ascending=[False], inplace=True) 

# Afficher uniquement certaines colonnes
#dataX = data_csv.filter(items=['elapsed', 'label'])

#récupérer uniquement où la loss est < à X et ou kernel = (3,3) par exemple
#dataX = data_csv.loc[(data_csv['elapsed'] > 700) & (data_csv['threadName'].str.contains('Thread Group 1-2'))]
#dataX

#pd.set_option('display.max_rows', data3.shape[0]+1) #nombre de row max à afficher
#data_csv = pd.read_csv(main_directory+"\\
_20200116-204456\\recap.csv")
#data_csv.head()

In [ ]:
image = pyplot.imread("CONVNETS_20200113-1951\\logs1\\indiv1_plot.png")
pyplot.imshow(image)

### TEST CODE

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.datasets import fashion_mnist

def show_first_samples(x_train, y_train):
    plt.imshow(x_train[0])
    print(y_train[0])
    plt.show()
    plt.imshow(x_train[1])
    print(y_train[1])
    plt.show()
    plt.imshow(x_train[2])
    print(y_train[2])
    plt.show()
    plt.imshow(x_train[3])
    print(y_train[3])
    plt.show()


class PlotRealLossAndAccuracyOnTrainCallback(Callback):

    def __init__(self, x_train, y_train):
        super().__init__()
        self.x_train = x_train
        self.y_train = y_train

    def on_epoch_end(self, epoch, logs=None):
        (loss, acc) = self.model.evaluate(self.x_train, self.y_train, batch_size = 8192)
        print(f"Real Loss on train : {loss}")
        print(f"Real Acc on train : {acc}")


def create_model(depth: int = 2, use_skip_connections: bool = True):
    input_layer = Input((28, 28))
    flatten_layer_output = Flatten(name="flatten")(input_layer)

    penultimate_output = None
    last_output = flatten_layer_output

    for i in range(depth):
        if penultimate_output is not None and use_skip_connections:
            add_output = Add(name=f"Add_{i}")([last_output, penultimate_output])
            penultimate_output = add_output
            last_output = Dense(784, activation=linear, name=f"Dense_{i}", kernel_regularizer=L1L2(l2=0.001 / depth))(
                add_output)
            # last_output = BatchNormalization()(last_output)
            last_output = Activation(activation=relu, name=f"Activation_{i}")(last_output)
        else:
            penultimate_output = last_output
            last_output = Dense(784, activation=linear, name=f"Dense_{i}", kernel_regularizer=L1L2(l2=0.001 / depth))(
                last_output)
            # last_output = BatchNormalization()(last_output)
            last_output = Activation(activation=relu, name=f"Activation_{i}")(last_output)

    if use_skip_connections:
        last_output = Add(name=f"Add_output")([last_output, penultimate_output])

    output_tensor = Dense(10, activation=softmax, name=f"Dense_output", kernel_regularizer=L1L2(l2=0.001 / depth))(
        last_output)
    model = Model(input_layer, output_tensor)

    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=Adam(),
                  metrics=['sparse_categorical_accuracy'])
    return model


if __name__ == "__main__":
    (x_train, y_train), (x_val, y_val) = cifar10.load_data()
    x_train = x_train / 255.0
    x_val = x_val / 255.0
    #show_first_samples(x_train, y_train)
    model = create_model(use_skip_connections=False)
    print(model.summary())
    plot_model(model, "residual_dense.png")
    model.fit(x_train, y_train, validation_data=(x_val, y_val),
              epochs=100,
              batch_size=8192,
              callbacks=[PlotRealLossAndAccuracyOnTrainCallback(x_train, y_train)])